In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

In [2]:
# Downloading the MNIST training and testing datasets
train_dataset = torchvision.datasets.MNIST(root='data/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='data/', 
                                          train=False, 
                                          transform=transforms.ToTensor())

In [3]:
# Setting Hyper-parameters 
input_size = 784 # 28x28
hidden_size = 500
num_classes = 10 
num_epochs = 10
batch_size = 100
learning_rate = 0.001

In [4]:
# Data loader: loads batch_size of data on each iteration
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# examples = iter(train_loader)
# example_data, example_targets = next(examples)

# for i in range(6):
#     plt.subplot(2,3,i+1)
#     plt.imshow(example_data[i][0], cmap='gray')
# plt.show()

In [5]:
# Setting up the Model

# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        return out

model = NeuralNet(input_size, hidden_size, num_classes)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()

# The Adam optimizer is similar to SGD but with an adaptive learning rate that uses mean and variance of the gradient
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [6]:
# Train the model
n_total_steps = len(train_loader)

for epoch in range(num_epochs):
    
    for i, (images, labels) in enumerate(train_loader):  
        # original shape: [100, 1, 28, 28], resized shape: [100, 784]. Setting up each image for the neural network
        images = images.reshape(-1, 28*28)
        
        # Forward pass and loss calculation
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backpropagation for computing gradients and storing them in the parameters' grad attribute
        loss.backward()

        # Take 1 step of the optimizer(SGD in this case), updates the parameter values by subtracting lr*gradient 
        optimizer.step()

        # reset the gradient for the next iteration
        optimizer.zero_grad()
        
        if (i+1) % 100 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [100/600], Loss: 0.2032
Epoch [1/10], Step [200/600], Loss: 0.2205
Epoch [1/10], Step [300/600], Loss: 0.1336
Epoch [1/10], Step [400/600], Loss: 0.1175
Epoch [1/10], Step [500/600], Loss: 0.0878
Epoch [1/10], Step [600/600], Loss: 0.0714
Epoch [2/10], Step [100/600], Loss: 0.0682
Epoch [2/10], Step [200/600], Loss: 0.0654
Epoch [2/10], Step [300/600], Loss: 0.0415
Epoch [2/10], Step [400/600], Loss: 0.0510
Epoch [2/10], Step [500/600], Loss: 0.0349
Epoch [2/10], Step [600/600], Loss: 0.0862
Epoch [3/10], Step [100/600], Loss: 0.0213
Epoch [3/10], Step [200/600], Loss: 0.0224
Epoch [3/10], Step [300/600], Loss: 0.0368
Epoch [3/10], Step [400/600], Loss: 0.0218
Epoch [3/10], Step [500/600], Loss: 0.0292
Epoch [3/10], Step [600/600], Loss: 0.0616
Epoch [4/10], Step [100/600], Loss: 0.0099
Epoch [4/10], Step [200/600], Loss: 0.0772
Epoch [4/10], Step [300/600], Loss: 0.0987
Epoch [4/10], Step [400/600], Loss: 0.0068
Epoch [4/10], Step [500/600], Loss: 0.0084
Epoch [4/10

In [7]:
# Test the model: we don't need to compute gradients
with torch.no_grad():
    n_correct = 0
    n_samples = len(test_loader.dataset)

    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)

        outputs = model(images)

        # max returns (output_value ,index)
        _, predicted = torch.max(outputs, 1)
        n_correct += (predicted == labels).sum().item()

    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 10000 test images: 98.28 %
